In [2]:
import os
os.chdir('../')

In [5]:
from utils import team_utils
import pickle
import pandas as pd
import numpy as np
from dspl.db import TeraDataDB

import os
import sys
import pickle
import pandas as pd
import warnings
import gc

from dspl.models import classifiers
warnings.simplefilter("ignore")
import os
import teradatasql
from joblib import Parallel, delayed
import multiprocessing
from functools import partial


from teradata_tools import td_import

In [ ]:
batch_count = 20  # Кол-во процессов и одновременно заливаемы батчей


def get_proc_id(i):  # Получаем номер процесса
    print(multiprocessing.current_process())
    return multiprocessing.current_process()._identity[0]


def create_table(table_name, proc_id):  # Создаем собственную таблицу под кажды процесс

    table_name_id = table_name + str(proc_id)

    tdHost = ''
    tdUser = ''
    tdPwd = ''

    connection = teradatasql.connect(
        host=tdHost,
        user=tdUser,
        password=tdPwd
    )

    cursor_create = connection.cursor()

    cursor_create.execute(f'''
    CREATE MULTISET TABLE {table_name_id} ,NO FALLBACK ,
         NO BEFORE JOURNAL,
         NO AFTER JOURNAL,
         CHECKSUM = DEFAULT,
         DEFAULT MERGEBLOCKRATIO,
         MAP = TD_MAP2
         (
            EPK_ID BIGINT,
            "0" DECIMAL(5,4),
            "1" DECIMAL(5,4),
            "2" DECIMAL(5,4),
            "3" DECIMAL(5,4),
            "4" DECIMAL(5,4),
            "5" DECIMAL(5,4),
            "6" DECIMAL(5,4),
            "7" DECIMAL(5,4),
            "8" DECIMAL(5,4)
)
    PRIMARY INDEX ( EPK_ID );
    ''')

    cursor_create.execute(f'''
    grant all on {table_name_id} to "";
    ''')
    
    connection.close()
    del connection
    print('Create: ', table_name_id)
    pass 


def create_conect(i):  # открываем конекшены
    if i % 2 == 0:
        tdHost = ''
        tdUser = ''
        tdPwd = ''

        connection = teradatasql.connect(
            host=tdHost,
            user=tdUser,
            password=tdPwd
        )
        print('get_connect: ', tdUser,i)
    else:
        tdHost = ''
        tdUser = ''
        tdPwd = '' 

        connection = teradatasql.connect(
            host=tdHost,
            user=tdUser,
            password=tdPwd
        )
        print('get_connect: ', tdUser,i)
    return connection


def close_conect(connection):  # Закрываем конекшены
    proc_id = multiprocessing.current_process()._identity[0]
    connection.close()
    print('Close_con: ', proc_id)
    pass


def drop_table(table_name,connection):  # Удаляем временные талицы
    proc_id = multiprocessing.current_process()._identity[0]
    table_name = table_name + str(proc_id)
    cursor_drop = connection.cursor()
    
    cursor_drop.execute(f'''
    DROP TABLE {table_name}
    ''')
    
    print('Drop: ', table_name)
    pass 


def merge(proc_id,table_name):  # Соединяем временные таблицы
    union = str()
    for n in proc_id:
        table_name_id = table_name + str(n)
        union += f'(SELECT * FROM {table_name_id}) UNION ALL '

    tdHost = ''
    tdUser = ''
    tdPwd = '' 

    connection_merge = teradatasql.connect(
        host=tdHost,
        user=tdUser,
        password=tdPwd
    )

    cursor_merge = connection_merge.cursor()
    
    cursor_merge.execute(f'''
    DELETE FROM {table_name[:-1]} 
    ''')

    cursor_merge.execute(f'''
    INSERT INTO {table_name[:-1]} (epk_id, "0", "1", "2", "3", "4", "5", "6", "7", "8")
    SELECT agr.epk_id, agr."0", agr."1", agr."2", agr."3", agr."4", agr."5", agr."6", agr."7", agr."8" 
    FROM ({union[:-10]}) as agr
    ''')
    connection_merge.close()
    
    print(f'Merage to {table_name[:-1]} DONE')
    pass


def csv_to_td_amazing(connection, path, table_name, batches): 
    '''
    Функция для параллельноной перекладки батчей в teradata


    Parameters
        ----------
        connection: teradata connection (list)
            Список открытых соединений

        path: str
            Путь до папки с батчами

        table_name: str
            Название временных таблиц

        batches: str (list)
            Список из назания батчей, по которым происходит параллельная выгрузка

        Returns
        -------
        mod_enc: model, lenght = 9
            На ваход подается список моделей, по которым происходит параллельная итерация
    '''
    
    proc_id = multiprocessing.current_process()._identity[0]
    df = pd.read_csv(path + batches, dtype={'epk_id': 'int64'})
    df = df[df['epk_id'] > 0]
    print(f'{batches} shape: {df.shape} {proc_id}')
    tup = [tuple([int(x[0]), float(x[1]), float(x[2]), float(x[3]), float(x[4]), float(x[5]),
                  float(x[6]), float(x[7]), float(x[8]), float(x[9])])
           for x in df.to_records(index=False)]
    conn = connection[(proc_id - 1) % len(connection)]
    cursor = conn.cursor()
    table_name = table_name + str(proc_id)
    stmt = f'''INSERT INTO {table_name} (epk_id, "0", "1", "2", "3", "4", "5", "6", "7", "8")
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)'''
    print(f'INSERTING: {batches} to {table_name}, {proc_id}')
    cursor.executemany(stmt, tup)
    print('insert_done: ',proc_id)
    pass

In [ ]:
# load
if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=batch_count)
    proc_id = pool.map(get_proc_id, range(batch_count))

    if len(set(proc_id)) == batch_count:

        path = 'scoring_data/Data/batch/'
        table_name = f''

        f_careate = partial(create_table, table_name)
        pool.map(f_careate, proc_id)

        connection = pool.map(create_conect, range(batch_count))

        f_insert = partial(csv_to_td_amazing, connection, path, table_name)
        batches = os.listdir(path)
        batches = [bat for bat in batches if bat[0] != "."]
        %time pool.map(f_insert, batches)

        merge(proc_id, table_name)

        f_drop = partial(drop_table, table_name)
        pool.map(f_drop, connection)

        pool.map(close_conect, connection)
        del connection

    pool.close()
    pool.join()
    del pool